# Notebook 6: KV Cache Mechanics

## Inference Engineering Course

---

### What You'll Learn

The **KV Cache** (Key-Value Cache) is one of the most important optimizations in LLM inference. Without it, generating each new token would require recomputing attention over the *entire* sequence from scratch. With it, we only compute attention for the *new* token, reusing previously computed key and value vectors.

In this notebook, we will:

1. **Understand why naive autoregressive generation is slow** - the redundant computation problem
2. **Implement attention with and without KV cache** from scratch
3. **Measure computational savings** in FLOPs and wall-clock time
4. **Visualize cache growth** over sequence length
5. **Calculate memory requirements** for different model sizes
6. **Benchmark generation speed** with and without cache

### Prerequisites
- Understanding of attention mechanism basics (Q, K, V matrices)
- Basic PyTorch
- Linear algebra fundamentals

### Runtime
- **No GPU required** - all examples use small dimensions

---

## 1. Setup

In [ ]:
!pip install torch matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import time
from typing import Optional, Tuple

torch.manual_seed(42)
np.random.seed(42)

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

## 2. The Redundant Computation Problem

### How Autoregressive Generation Works

Language models generate text **one token at a time**. At each step:
1. The model takes all previous tokens as input
2. Computes attention across the full sequence
3. Outputs the next token
4. Appends that token and repeats

### The Problem

Consider generating a sequence of length $n$. At step $t$, naive attention computes:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V$$

where $Q$, $K$, and $V$ are derived from ALL tokens $[1, 2, ..., t]$.

But here's the key insight: **the K and V vectors for tokens 1 through t-1 are exactly the same as they were at step t-1!** Only the new token $t$ produces a new K and V.

Without caching, we redundantly recompute K and V for all previous tokens at every single step.

Let's quantify this waste.

In [ ]:
# Visualize the redundant computation
seq_length = 8

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Without KV Cache: full computation at each step
for step in range(1, seq_length + 1):
    for pos in range(step):
        color = '#d62728' if pos < step - 1 else '#2ca02c'
        alpha = 0.3 if pos < step - 1 else 0.9
        ax1.add_patch(plt.Rectangle((pos, seq_length - step), 1, 1,
                      facecolor=color, edgecolor='white', alpha=alpha, linewidth=2))

ax1.set_xlim(0, seq_length)
ax1.set_ylim(0, seq_length)
ax1.set_xlabel('Token Position (K/V computation)', fontsize=12)
ax1.set_ylabel('Generation Step', fontsize=12)
ax1.set_title('Without KV Cache\n(Red = Redundant Recomputation)', fontsize=13, fontweight='bold')
ax1.set_yticks(np.arange(0.5, seq_length, 1))
ax1.set_yticklabels(range(seq_length, 0, -1))
ax1.set_xticks(np.arange(0.5, seq_length, 1))
ax1.set_xticklabels(range(1, seq_length + 1))

# With KV Cache: only new token computed
for step in range(1, seq_length + 1):
    # Cached positions (gray)
    for pos in range(step - 1):
        ax2.add_patch(plt.Rectangle((pos, seq_length - step), 1, 1,
                      facecolor='#cccccc', edgecolor='white', alpha=0.5, linewidth=2))
    # New computation (green)
    ax2.add_patch(plt.Rectangle((step - 1, seq_length - step), 1, 1,
                  facecolor='#2ca02c', edgecolor='white', alpha=0.9, linewidth=2))

ax2.set_xlim(0, seq_length)
ax2.set_ylim(0, seq_length)
ax2.set_xlabel('Token Position (K/V computation)', fontsize=12)
ax2.set_ylabel('Generation Step', fontsize=12)
ax2.set_title('With KV Cache\n(Gray = Cached, Green = New)', fontsize=13, fontweight='bold')
ax2.set_yticks(np.arange(0.5, seq_length, 1))
ax2.set_yticklabels(range(seq_length, 0, -1))
ax2.set_xticks(np.arange(0.5, seq_length, 1))
ax2.set_xticklabels(range(1, seq_length + 1))

# Legend
red_patch = mpatches.Patch(color='#d62728', alpha=0.5, label='Redundant (recomputed)')
green_patch = mpatches.Patch(color='#2ca02c', alpha=0.9, label='New computation')
gray_patch = mpatches.Patch(color='#cccccc', alpha=0.5, label='Cached (free)')
fig.legend(handles=[red_patch, green_patch, gray_patch], loc='lower center',
           ncol=3, fontsize=11, bbox_to_anchor=(0.5, -0.05))

plt.tight_layout()
plt.show()

# Count operations
without_cache = sum(range(1, seq_length + 1))
with_cache = seq_length
print(f"K/V projections without cache: {without_cache}")
print(f"K/V projections with cache:    {with_cache}")
print(f"Savings: {without_cache - with_cache} projections ({(1 - with_cache/without_cache)*100:.1f}% reduction)")

## 3. Implementing Attention Without KV Cache

Let's first implement standard self-attention that recomputes everything from scratch. This is the baseline we want to optimize.

Recall the attention formula:
$$Q = XW_Q, \quad K = XW_K, \quad V = XW_V$$
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V$$

In [ ]:
class SelfAttentionNoCache(nn.Module):
    """Standard self-attention WITHOUT KV cache.
    
    At each generation step, this module:
    1. Projects ALL tokens to Q, K, V
    2. Computes full attention matrix
    3. Returns output for the last token
    
    This is wasteful during autoregressive generation because
    K and V for previous tokens are recomputed unnecessarily.
    """
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model, bias=False)
    
    def forward(self, x):
        """Forward pass: compute attention over full sequence.
        
        Args:
            x: Input tensor of shape (batch, seq_len, d_model)
        Returns:
            Output tensor of same shape
        """
        batch_size, seq_len, _ = x.shape
        
        # Project ALL tokens to Q, K, V
        Q = self.W_q(x)  # (batch, seq_len, d_model)
        K = self.W_k(x)  # (batch, seq_len, d_model)
        V = self.W_v(x)  # (batch, seq_len, d_model)
        
        # Reshape for multi-head attention
        Q = Q.view(batch_size, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        K = K.view(batch_size, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        V = V.view(batch_size, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        # Shape: (batch, n_heads, seq_len, d_head)
        
        # Compute attention scores
        scale = self.d_head ** 0.5
        scores = torch.matmul(Q, K.transpose(-2, -1)) / scale
        
        # Apply causal mask (prevent attending to future tokens)
        causal_mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1).bool()
        scores = scores.masked_fill(causal_mask.unsqueeze(0).unsqueeze(0), float('-inf'))
        
        # Softmax and weighted sum
        attn_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attn_weights, V)
        
        # Reshape back
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        return self.W_o(output)

# Test it
d_model = 64
n_heads = 4
attn_no_cache = SelfAttentionNoCache(d_model, n_heads)

x = torch.randn(1, 5, d_model)  # batch=1, seq_len=5
output = attn_no_cache(x)
print(f"Input shape:  {x.shape}")
print(f"Output shape: {output.shape}")

## 4. Implementing Attention WITH KV Cache

Now let's implement the optimized version. The key insight is:

**During generation, we only need Q for the NEW token, but K and V for ALL tokens (including past ones).**

So we:
1. Compute Q, K, V only for the new token
2. Append the new K, V to the cache
3. Use full cached K, V for attention computation
4. Return only the output for the new token

This changes attention from $O(n^2)$ per step to $O(n)$ per step for the projection.

In [ ]:
class SelfAttentionWithCache(nn.Module):
    """Self-attention WITH KV cache for efficient generation.
    
    During the prefill phase (processing the prompt), this works
    like normal attention and populates the cache.
    
    During the decode phase (generating tokens), it only computes
    Q/K/V for the new token and reuses cached K/V from previous steps.
    """
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_model // n_heads
        
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model, bias=False)
    
    def forward(self, x, kv_cache=None):
        """Forward pass with optional KV cache.
        
        Args:
            x: Input tensor. 
               - Prefill: (batch, seq_len, d_model)
               - Decode:  (batch, 1, d_model) - just the new token
            kv_cache: Tuple of (cached_K, cached_V) or None for first call
        
        Returns:
            output: Attention output
            new_kv_cache: Updated (K, V) cache
        """
        batch_size, seq_len, _ = x.shape
        
        # Compute Q, K, V for the input (could be full prompt or single token)
        Q = self.W_q(x)
        K_new = self.W_k(x)
        V_new = self.W_v(x)
        
        # Reshape for multi-head attention
        Q = Q.view(batch_size, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        K_new = K_new.view(batch_size, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        V_new = V_new.view(batch_size, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        
        # Append to cache or create new cache
        if kv_cache is not None:
            K_cached, V_cached = kv_cache
            # Concatenate new K, V with cached K, V
            K = torch.cat([K_cached, K_new], dim=2)  # Concatenate along seq_len
            V = torch.cat([V_cached, V_new], dim=2)
        else:
            K = K_new
            V = V_new
        
        # Store updated cache
        new_kv_cache = (K, V)
        
        # Compute attention (Q attends to ALL K/V, including cached)
        total_len = K.shape[2]  # Full sequence length in cache
        scale = self.d_head ** 0.5
        scores = torch.matmul(Q, K.transpose(-2, -1)) / scale
        # scores shape: (batch, n_heads, seq_len, total_len)
        
        # Apply causal mask
        if seq_len > 1:  # Prefill phase: need full causal mask
            causal_mask = torch.triu(torch.ones(seq_len, total_len), diagonal=1).bool()
            scores = scores.masked_fill(causal_mask.unsqueeze(0).unsqueeze(0), float('-inf'))
        # In decode phase (seq_len=1), no masking needed - single query attends to all past
        
        attn_weights = F.softmax(scores, dim=-1)
        output = torch.matmul(attn_weights, V)
        
        # Reshape back
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)
        return self.W_o(output), new_kv_cache

# Test it
attn_with_cache = SelfAttentionWithCache(d_model, n_heads)

# Copy weights from the no-cache version for fair comparison
attn_with_cache.W_q.weight.data = attn_no_cache.W_q.weight.data.clone()
attn_with_cache.W_k.weight.data = attn_no_cache.W_k.weight.data.clone()
attn_with_cache.W_v.weight.data = attn_no_cache.W_v.weight.data.clone()
attn_with_cache.W_o.weight.data = attn_no_cache.W_o.weight.data.clone()

# Prefill with 3 tokens
x_prefill = x[:, :3, :]  # First 3 tokens
out_prefill, cache = attn_with_cache(x_prefill, kv_cache=None)
print(f"Prefill - Input: {x_prefill.shape}, Output: {out_prefill.shape}")
print(f"Cache K shape: {cache[0].shape}, Cache V shape: {cache[1].shape}")

# Decode token 4
x_token4 = x[:, 3:4, :]  # Just token 4
out_token4, cache = attn_with_cache(x_token4, kv_cache=cache)
print(f"\nDecode step 1 - Input: {x_token4.shape}, Output: {out_token4.shape}")
print(f"Cache K shape: {cache[0].shape}, Cache V shape: {cache[1].shape}")

# Decode token 5
x_token5 = x[:, 4:5, :]
out_token5, cache = attn_with_cache(x_token5, kv_cache=cache)
print(f"\nDecode step 2 - Input: {x_token5.shape}, Output: {out_token5.shape}")
print(f"Cache K shape: {cache[0].shape}, Cache V shape: {cache[1].shape}")

## 5. Verifying Correctness

The cached version should produce **identical** results to the non-cached version. Let's verify this. Any difference would indicate a bug.

In [ ]:
# Generate output with no-cache version (full sequence)
with torch.no_grad():
    output_no_cache = attn_no_cache(x)  # Full 5-token sequence

# Generate output with cache version (prefill + decode)
with torch.no_grad():
    # Prefill: process tokens 1-3
    out1, cache = attn_with_cache(x[:, :3, :], kv_cache=None)
    # Decode: process tokens 4 and 5 one at a time
    out2, cache = attn_with_cache(x[:, 3:4, :], kv_cache=cache)
    out3, cache = attn_with_cache(x[:, 4:5, :], kv_cache=cache)
    
    # Concatenate all outputs
    output_with_cache = torch.cat([out1, out2, out3], dim=1)

# Compare
max_diff = (output_no_cache - output_with_cache).abs().max().item()
mean_diff = (output_no_cache - output_with_cache).abs().mean().item()

print(f"Maximum absolute difference: {max_diff:.2e}")
print(f"Mean absolute difference:    {mean_diff:.2e}")
print(f"\nOutputs match: {max_diff < 1e-5}")

if max_diff < 1e-5:
    print("\nThe KV cache produces identical results to the naive approach.")
    print("This confirms our implementation is correct.")

## 6. Measuring Computational Savings

### FLOPs Analysis

Let's count the floating-point operations (FLOPs) for both approaches.

**Without KV cache** - generating $n$ tokens after a prompt of length $p$:
- At step $t$ (total sequence length $p + t$), we compute:
  - Q, K, V projections: $3 \times (p + t) \times d_{model}^2$ FLOPs
  - Attention scores: $(p + t)^2 \times d_{model}$ FLOPs

**With KV cache** - generating $n$ tokens after a prompt of length $p$:
- Prefill (once): $3 \times p \times d_{model}^2$ + $p^2 \times d_{model}$
- Each decode step $t$: $3 \times d_{model}^2$ + $(p + t) \times d_{model}$

In [ ]:
def compute_flops_no_cache(prompt_len, gen_len, d_model):
    """Total FLOPs for generation WITHOUT KV cache."""
    total_flops = 0
    for t in range(gen_len):
        seq_len = prompt_len + t
        # Q, K, V projections for all tokens
        proj_flops = 3 * seq_len * d_model * d_model * 2  # multiply-add
        # Attention score computation
        attn_flops = seq_len * seq_len * d_model * 2
        # Output projection
        out_flops = seq_len * d_model * d_model * 2
        total_flops += proj_flops + attn_flops + out_flops
    return total_flops

def compute_flops_with_cache(prompt_len, gen_len, d_model):
    """Total FLOPs for generation WITH KV cache."""
    # Prefill phase: process entire prompt
    prefill_proj = 3 * prompt_len * d_model * d_model * 2
    prefill_attn = prompt_len * prompt_len * d_model * 2
    prefill_out = prompt_len * d_model * d_model * 2
    total_flops = prefill_proj + prefill_attn + prefill_out
    
    # Decode phase: one token at a time
    for t in range(gen_len):
        seq_len = prompt_len + t
        # Q, K, V projections for ONE token only
        proj_flops = 3 * 1 * d_model * d_model * 2
        # Attention: one query against all keys
        attn_flops = 1 * seq_len * d_model * 2
        # Output projection for one token
        out_flops = 1 * d_model * d_model * 2
        total_flops += proj_flops + attn_flops + out_flops
    return total_flops

# Compare FLOPs for different sequence lengths
d_model = 4096  # Typical for 7B model
prompt_len = 512
gen_lengths = [32, 64, 128, 256, 512, 1024, 2048]

flops_no_cache = [compute_flops_no_cache(prompt_len, g, d_model) for g in gen_lengths]
flops_with_cache = [compute_flops_with_cache(prompt_len, g, d_model) for g in gen_lengths]
speedup = [f1/f2 for f1, f2 in zip(flops_no_cache, flops_with_cache)]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(gen_lengths, [f/1e12 for f in flops_no_cache], 'r-o', label='Without Cache', linewidth=2)
ax1.plot(gen_lengths, [f/1e12 for f in flops_with_cache], 'g-o', label='With Cache', linewidth=2)
ax1.set_xlabel('Generated Tokens')
ax1.set_ylabel('Total FLOPs (TFLOPs)')
ax1.set_title(f'Computation Cost (d_model={d_model}, prompt={prompt_len})')
ax1.legend()
ax1.set_yscale('log')

ax2.bar(range(len(gen_lengths)), speedup, color='steelblue', alpha=0.8)
ax2.set_xticks(range(len(gen_lengths)))
ax2.set_xticklabels(gen_lengths)
ax2.set_xlabel('Generated Tokens')
ax2.set_ylabel('Speedup Factor (x)')
ax2.set_title('KV Cache Speedup Factor')

for i, s in enumerate(speedup):
    ax2.text(i, s + 0.5, f'{s:.1f}x', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Wall-Clock Benchmarking

FLOPs are theoretical - let's measure actual wall-clock time. We'll simulate autoregressive generation with both approaches.

In [ ]:
def benchmark_generation(attn_module, d_model, prompt_len, gen_len, use_cache=False):
    """Benchmark autoregressive generation."""
    # Create random embeddings (simulating model output)
    prompt = torch.randn(1, prompt_len, d_model)
    
    start = time.perf_counter()
    
    with torch.no_grad():
        if use_cache:
            # Prefill
            _, cache = attn_module(prompt, kv_cache=None)
            # Decode one token at a time
            for t in range(gen_len):
                new_token = torch.randn(1, 1, d_model)
                _, cache = attn_module(new_token, kv_cache=cache)
        else:
            # Without cache: recompute everything each step
            sequence = prompt
            for t in range(gen_len):
                _ = attn_module(sequence)
                new_token = torch.randn(1, 1, d_model)
                sequence = torch.cat([sequence, new_token], dim=1)
    
    elapsed = time.perf_counter() - start
    return elapsed

# Benchmark
d_model = 128  # Smaller for CPU benchmarking
n_heads = 4
prompt_len = 32
gen_lengths = [10, 20, 50, 100, 150, 200]

attn_nc = SelfAttentionNoCache(d_model, n_heads)
attn_wc = SelfAttentionWithCache(d_model, n_heads)

# Copy weights
attn_wc.W_q.weight.data = attn_nc.W_q.weight.data.clone()
attn_wc.W_k.weight.data = attn_nc.W_k.weight.data.clone()
attn_wc.W_v.weight.data = attn_nc.W_v.weight.data.clone()
attn_wc.W_o.weight.data = attn_nc.W_o.weight.data.clone()

times_no_cache = []
times_with_cache = []

print(f"Benchmarking (d_model={d_model}, n_heads={n_heads}, prompt={prompt_len})")
print("=" * 60)

for gen_len in gen_lengths:
    t_nc = benchmark_generation(attn_nc, d_model, prompt_len, gen_len, use_cache=False)
    t_wc = benchmark_generation(attn_wc, d_model, prompt_len, gen_len, use_cache=True)
    times_no_cache.append(t_nc)
    times_with_cache.append(t_wc)
    speedup = t_nc / t_wc
    print(f"Gen {gen_len:>3d} tokens: No cache={t_nc:.3f}s, With cache={t_wc:.3f}s, Speedup={speedup:.2f}x")

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(gen_lengths, times_no_cache, 'r-o', label='Without Cache', linewidth=2, markersize=8)
ax1.plot(gen_lengths, times_with_cache, 'g-o', label='With Cache', linewidth=2, markersize=8)
ax1.set_xlabel('Generated Tokens')
ax1.set_ylabel('Wall-Clock Time (seconds)')
ax1.set_title('Generation Time: With vs Without KV Cache')
ax1.legend()

speedups = [t1/t2 for t1, t2 in zip(times_no_cache, times_with_cache)]
ax2.plot(gen_lengths, speedups, 'b-o', linewidth=2, markersize=8)
ax2.set_xlabel('Generated Tokens')
ax2.set_ylabel('Speedup Factor')
ax2.set_title('Wall-Clock Speedup from KV Cache')
ax2.axhline(y=1, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 8. KV Cache Memory Analysis

The KV cache trades **memory** for **compute**. Let's analyze how much memory it requires.

### Memory Formula

For each layer, the KV cache stores:
- **K**: shape `(batch, n_heads, seq_len, d_head)`
- **V**: shape `(batch, n_heads, seq_len, d_head)`

Total KV cache memory per layer:
$$\text{Memory} = 2 \times \text{batch} \times n_{\text{heads}} \times \text{seq\_len} \times d_{\text{head}} \times \text{bytes\_per\_element}$$

For a full model with $L$ layers:
$$\text{Total KV Memory} = 2 \times L \times \text{batch} \times n_{\text{heads}} \times \text{seq\_len} \times d_{\text{head}} \times \text{bytes}$$

In [ ]:
def kv_cache_memory_gb(n_layers, n_heads, d_head, seq_len, batch_size=1, bytes_per_elem=2):
    """Calculate KV cache memory in GB.
    
    Args:
        n_layers: Number of transformer layers
        n_heads: Number of attention heads (for KV, may differ from Q heads)
        d_head: Dimension per head
        seq_len: Sequence length (context window used)
        batch_size: Batch size
        bytes_per_elem: Bytes per element (2 for FP16, 4 for FP32)
    """
    # 2 for K and V
    total_bytes = 2 * n_layers * batch_size * n_heads * seq_len * d_head * bytes_per_elem
    return total_bytes / (1024 ** 3)  # Convert to GB

# Real model configurations
models = {
    'LLaMA-7B':  {'n_layers': 32, 'n_heads': 32, 'd_head': 128, 'params': '7B'},
    'LLaMA-13B': {'n_layers': 40, 'n_heads': 40, 'd_head': 128, 'params': '13B'},
    'LLaMA-70B': {'n_layers': 80, 'n_heads': 64, 'd_head': 128, 'params': '70B'},
    'GPT-3 175B': {'n_layers': 96, 'n_heads': 96, 'd_head': 128, 'params': '175B'},
}

seq_lengths = [512, 1024, 2048, 4096, 8192, 16384, 32768]

print(f"KV Cache Memory (FP16, batch_size=1)")
print("=" * 80)
print(f"{'Model':<14s}", end="")
for s in seq_lengths:
    print(f"{'  ' + str(s):>8s}", end="")
print()
print("-" * 80)

for name, config in models.items():
    print(f"{name:<14s}", end="")
    for s in seq_lengths:
        mem = kv_cache_memory_gb(config['n_layers'], config['n_heads'],
                                 config['d_head'], s, bytes_per_elem=2)
        if mem < 1:
            print(f"{mem*1024:>6.0f}MB", end="")
        else:
            print(f"{mem:>6.1f}GB", end="")
    print()

In [ ]:
# Visualize KV cache memory growth
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

for (name, config), color in zip(models.items(), colors):
    mems = [kv_cache_memory_gb(config['n_layers'], config['n_heads'],
                                config['d_head'], s, bytes_per_elem=2)
            for s in seq_lengths]
    ax1.plot(seq_lengths, mems, 'o-', label=name, color=color, linewidth=2, markersize=6)

ax1.set_xlabel('Sequence Length')
ax1.set_ylabel('KV Cache Memory (GB)')
ax1.set_title('KV Cache Memory vs Sequence Length (FP16, batch=1)')
ax1.legend()
ax1.set_xscale('log', base=2)
ax1.set_yscale('log', base=2)

# GPU memory context lines
for gpu_mem, gpu_name in [(24, 'RTX 4090'), (40, 'A100 40GB'), (80, 'A100 80GB')]:
    ax1.axhline(y=gpu_mem, color='gray', linestyle=':', alpha=0.4)
    ax1.text(seq_lengths[0], gpu_mem * 1.1, gpu_name, fontsize=8, color='gray')

# Batch size scaling
batch_sizes = [1, 2, 4, 8, 16, 32, 64, 128]
seq_len = 4096

for (name, config), color in zip(models.items(), colors):
    mems = [kv_cache_memory_gb(config['n_layers'], config['n_heads'],
                                config['d_head'], seq_len, batch_size=b, bytes_per_elem=2)
            for b in batch_sizes]
    ax2.plot(batch_sizes, mems, 'o-', label=name, color=color, linewidth=2, markersize=6)

ax2.set_xlabel('Batch Size')
ax2.set_ylabel('KV Cache Memory (GB)')
ax2.set_title(f'KV Cache Memory vs Batch Size (seq_len={seq_len}, FP16)')
ax2.legend()

for gpu_mem, gpu_name in [(24, 'RTX 4090'), (40, 'A100 40GB'), (80, 'A100 80GB')]:
    ax2.axhline(y=gpu_mem, color='gray', linestyle=':', alpha=0.4)
    ax2.text(batch_sizes[0], gpu_mem * 1.05, gpu_name, fontsize=8, color='gray')

plt.tight_layout()
plt.show()

## 9. Prefill vs Decode: Two Phases of Generation

LLM inference has two distinct phases:

### Phase 1: Prefill (Prompt Processing)
- Process the entire prompt in parallel
- Compute K, V for all prompt tokens
- Store them in the KV cache
- **Compute-bound**: large matrix multiplications, good GPU utilization
- Time complexity: $O(n^2 \cdot d)$ where $n$ = prompt length

### Phase 2: Decode (Token Generation)
- Process one token at a time
- Compute Q for the new token only
- Attend to all cached K, V
- **Memory-bound**: mostly reading from KV cache, poor GPU utilization
- Time complexity per step: $O(n \cdot d)$

This distinction is critical for optimization. Let's visualize the difference.

In [ ]:
# Visualize prefill vs decode phases
fig, ax = plt.subplots(1, 1, figsize=(14, 6))

prompt_len = 100
gen_len = 50

# Simulate time per token (simplified model)
d_model = 4096
prefill_time = prompt_len * d_model * d_model / 1e12  # Normalized compute

# Decode times increase linearly as cache grows
decode_times = []
for t in range(gen_len):
    total_seq = prompt_len + t
    # Decode is memory-bound: time scales with cache reads
    decode_time = total_seq * d_model / 1e12 + d_model * d_model / 1e12
    decode_times.append(decode_time)

# Plot
x_positions = list(range(gen_len + 1))
times = [prefill_time] + decode_times

colors_bar = ['#d62728'] + ['#2ca02c'] * gen_len
ax.bar(x_positions, times, color=colors_bar, alpha=0.7, width=0.8)

ax.axvline(x=0.5, color='black', linestyle='--', alpha=0.5)
ax.text(0, max(times) * 0.9, 'PREFILL\n(parallel)', ha='center', fontsize=11,
        fontweight='bold', color='#d62728')
ax.text(gen_len // 2, max(times) * 0.9, 'DECODE\n(sequential, one token at a time)',
        ha='center', fontsize=11, fontweight='bold', color='#2ca02c')

ax.set_xlabel('Generation Step')
ax.set_ylabel('Relative Computation Time')
ax.set_title('Prefill vs Decode Phases in LLM Inference')

# Annotate key insight
ax.annotate('Prefill: compute-bound\n(large batch matmul)',
            xy=(0, prefill_time), xytext=(10, prefill_time * 1.1),
            arrowprops=dict(arrowstyle='->', color='gray'),
            fontsize=9, color='gray')

ax.annotate('Decode time grows as\nKV cache gets larger',
            xy=(gen_len - 1, decode_times[-1]),
            xytext=(gen_len - 15, decode_times[-1] * 1.5),
            arrowprops=dict(arrowstyle='->', color='gray'),
            fontsize=9, color='gray')

plt.tight_layout()
plt.show()

## 10. Cache Growth Visualization

Let's create a detailed visualization showing how the KV cache grows token by token during generation.

In [ ]:
# Detailed cache growth visualization
n_layers = 4  # Simplified model
n_heads = 4
d_head = 32
max_seq = 20

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Cache size per layer
ax = axes[0][0]
seq_range = range(1, max_seq + 1)
for layer in range(n_layers):
    cache_sizes = [2 * n_heads * s * d_head * 2 / 1024 for s in seq_range]  # KB, FP16
    ax.fill_between(seq_range, 
                     [layer * c for c in cache_sizes],
                     [(layer + 1) * c for c in cache_sizes],
                     alpha=0.7, label=f'Layer {layer}')
ax.set_xlabel('Sequence Length')
ax.set_ylabel('Cumulative KV Cache Size (KB)')
ax.set_title('KV Cache Growth: Stacked by Layer')
ax.legend()

# Plot 2: Total memory breakdown
ax = axes[0][1]
seq_lens = [64, 128, 256, 512, 1024]
model_config = {'n_layers': 32, 'n_heads': 32, 'd_head': 128}

k_memory = [model_config['n_layers'] * model_config['n_heads'] * s * model_config['d_head'] * 2 / (1024**3) for s in seq_lens]
v_memory = k_memory.copy()  # Same size for V

x_pos = np.arange(len(seq_lens))
ax.bar(x_pos - 0.15, k_memory, 0.3, label='K Cache', color='#1f77b4')
ax.bar(x_pos + 0.15, v_memory, 0.3, label='V Cache', color='#ff7f0e')
ax.set_xticks(x_pos)
ax.set_xticklabels(seq_lens)
ax.set_xlabel('Sequence Length')
ax.set_ylabel('Memory (GB)')
ax.set_title('K vs V Cache Memory (LLaMA-7B, FP16)')
ax.legend()

# Plot 3: Proportion of GPU memory used by KV cache
ax = axes[1][0]
gpu_memory = 24  # RTX 4090
model_weight_mem = 14  # ~14GB for 7B model in FP16

kv_mems = [2 * model_config['n_layers'] * model_config['n_heads'] * s * model_config['d_head'] * 2 / (1024**3)
           for s in range(1, 4097)]
available = [gpu_memory - model_weight_mem] * len(kv_mems)

ax.fill_between(range(1, 4097), 0, kv_mems, alpha=0.5, color='#d62728', label='KV Cache')
ax.axhline(y=gpu_memory - model_weight_mem, color='black', linestyle='--',
           label=f'Available ({gpu_memory}-{model_weight_mem}={gpu_memory-model_weight_mem}GB)')
ax.set_xlabel('Sequence Length')
ax.set_ylabel('Memory (GB)')
ax.set_title(f'KV Cache vs Available Memory (RTX 4090, LLaMA-7B FP16)')
ax.legend()

# Find where KV cache exceeds available memory
max_seq_len = next((i for i, m in enumerate(kv_mems) if m > gpu_memory - model_weight_mem), len(kv_mems))
ax.axvline(x=max_seq_len, color='gray', linestyle=':', alpha=0.5)
ax.text(max_seq_len + 50, (gpu_memory - model_weight_mem) * 0.5,
        f'Max seq: ~{max_seq_len}', fontsize=10)

# Plot 4: Batch size vs max sequence length tradeoff
ax = axes[1][1]
available_mem = gpu_memory - model_weight_mem  # GB
batch_sizes = range(1, 33)

for model_name, config in [('LLaMA-7B', {'n_layers': 32, 'n_heads': 32, 'd_head': 128})]:
    max_seqs = []
    for bs in batch_sizes:
        mem_per_token = 2 * config['n_layers'] * config['n_heads'] * config['d_head'] * 2 / (1024**3) * bs
        max_seq = int(available_mem / mem_per_token)
        max_seqs.append(max_seq)
    ax.plot(batch_sizes, max_seqs, 'b-o', markersize=4, linewidth=2, label=model_name)

ax.set_xlabel('Batch Size')
ax.set_ylabel('Max Sequence Length')
ax.set_title(f'Batch Size vs Max Context (RTX 4090, {available_mem}GB available)')
ax.legend()

plt.tight_layout()
plt.show()

## 11. Optimizations: Reducing KV Cache Memory

The KV cache is often the memory bottleneck. Several techniques exist to reduce it:

| Technique | Memory Savings | Quality Impact |
|-----------|---------------|----------------|
| FP16 instead of FP32 | 2x | Negligible |
| INT8 KV cache | 4x vs FP32 | Small |
| Multi-Query Attention (MQA) | n_heads x | Small |
| Grouped-Query Attention (GQA) | n_heads/n_groups x | Very small |
| Sliding Window | seq_len/window x | Context-dependent |
| Token pruning | Variable | Task-dependent |

We'll explore MQA and GQA in detail in the next notebook. Here, let's visualize the impact of quantizing the KV cache.

In [ ]:
# Compare KV cache memory with different precision
model_config = {'n_layers': 32, 'n_heads': 32, 'd_head': 128}  # LLaMA-7B
seq_len = 4096

precisions = {
    'FP32 (4 bytes)': 4,
    'FP16 (2 bytes)': 2,
    'INT8 (1 byte)': 1,
    'INT4 (0.5 bytes)': 0.5,
}

fig, ax = plt.subplots(1, 1, figsize=(10, 6))

colors = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']
mem_values = []

for (name, bytes_elem), color in zip(precisions.items(), colors):
    mem = kv_cache_memory_gb(model_config['n_layers'], model_config['n_heads'],
                             model_config['d_head'], seq_len, bytes_per_elem=bytes_elem)
    mem_values.append(mem)

bars = ax.bar(range(len(precisions)), mem_values, color=colors, alpha=0.8)
ax.set_xticks(range(len(precisions)))
ax.set_xticklabels(precisions.keys(), rotation=15)
ax.set_ylabel('KV Cache Memory (GB)')
ax.set_title(f'KV Cache Memory at Different Precisions\n(LLaMA-7B, seq_len={seq_len}, batch=1)')

for i, (bar, mem) in enumerate(zip(bars, mem_values)):
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
            f'{mem:.2f} GB', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 12. Key Takeaways

### What We Learned

1. **KV Cache eliminates redundant computation** by storing previously computed Key and Value vectors

2. **Two-phase generation**:
   - **Prefill**: Process prompt in parallel, compute-bound
   - **Decode**: Generate one token at a time, memory-bound

3. **Memory-compute tradeoff**: KV cache saves massive compute but requires significant memory that grows linearly with sequence length

4. **Memory is the bottleneck** for long contexts and large batches

5. **Optimization directions**: Quantized KV cache, attention variants (MQA/GQA), sliding window attention

### The Numbers That Matter

For a 7B parameter model with 4096 context:
- FP16 KV cache: ~2 GB per request
- On a 24GB GPU: ~5 concurrent requests maximum
- With INT8 KV cache: ~1 GB per request, doubling throughput

---

## Exercises

### Exercise 1: Sliding Window KV Cache
Implement a sliding window KV cache that only keeps the most recent `window_size` tokens. Measure memory savings vs full cache.

In [ ]:
class SlidingWindowAttention(nn.Module):
    """Attention with sliding window KV cache.
    
    Only keeps the most recent window_size tokens in cache.
    This bounds memory usage regardless of sequence length.
    
    TODO: Implement this module
    Hint: Modify the forward method to truncate the cache
    when it exceeds window_size.
    """
    def __init__(self, d_model, n_heads, window_size=256):
        super().__init__()
        self.window_size = window_size
        # TODO: Add projection layers and implement forward
        pass
    
    def forward(self, x, kv_cache=None):
        # TODO: Implement attention with sliding window cache
        pass

### Exercise 2: Multi-Layer KV Cache
Extend the KV cache implementation to work with a stack of multiple attention layers. Track total memory usage across all layers.

In [ ]:
# TODO: Create a TransformerStack class that manages KV caches for multiple layers
# Track and report total memory usage
# Verify outputs match a non-cached implementation

### Exercise 3: Benchmark Analysis
Run the benchmark with different `d_model` sizes (64, 128, 256, 512) and plot how the KV cache speedup changes. At what point does the memory overhead of the cache become significant?

In [ ]:
# TODO: Sweep d_model values and benchmark
# Plot speedup vs d_model
# Analyze the compute-memory tradeoff

---

**Next up: Notebook 07 - Attention Variants (MHA, MQA, GQA)** where we'll explore different attention architectures that reduce KV cache memory while preserving quality.